In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
from torch.autograd import Function
import torch.optim as optim

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.metrics.functional import classification

import cv2
import geffnet
import math

from sklearn.manifold import TSNE
from scipy.spatial import ConvexHull
import inspect

import warnings
warnings.filterwarnings('ignore')

#model save
import pickle
from pickle import dump, load

import tidalUtl.PrpUtl as prp
import tidalUtl.EdaUtl as eda

In [2]:
#import sys
#sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

https://www.kaggle.com/tidalryoku/new-baseline-pytorch-moa/

# Version

__ver1__<br>
baseline：CV:0.01465 LB:0.01874<br>
__ver2__<br>
Hyperopt, 2Layer：CV:0.01460 LB:0.01869<br>
__ver3__<br>
3Layer：CV:0.01464 LB:0.01868<br>
__ver4__<br>
MLSMOTE baseline：CV:0.01476 LB:0.01978<br>
__ver5__<br>
2Layer,refactoring：CV:0.01476 LB:0.01869<br>
__ver6__<br>
rankGauss：CV:0.01456 LB:0.01865<br>
__ver7__<br>
labelSmoothing：CV:0.01502 LB:0.01859<br>

# Config

In [3]:
INPUT = "/home/tidal/ML_Data/MoA/lish-moa"
OUTPUT = "/home/tidal/ML_Data/MoA/output"
#INPUT = "/Users/hfuis/ML_Data/MoA/lish-moa"
#OUTPUT = "/Users/hfuis/ML_Data/MoA/output"

SUBMIT = OUTPUT + "/submittion/"
SAVEMODEL = OUTPUT + "/model/pytorchLightning_Efficientnet"

SAVEDEEPINSIGHT = OUTPUT + "/DeepInsightModel/"
SAVELOGSCALE = OUTPUT + "/LogScaler/"

In [4]:
#Loading
trainFeature = pd.read_csv(INPUT + '/train_features.csv')
testFeature = pd.read_csv(INPUT + '/test_features.csv')
trainTargetScored = pd.read_csv(INPUT + '/train_targets_scored.csv')
sample_submission = pd.read_csv(INPUT + '/sample_submission.csv')
drug = pd.read_csv(INPUT + '/train_drug.csv')

In [5]:
GENES = [col for col in trainFeature.columns if col.startswith('g-')] #gから始まる列名のセット
CELLS = [col for col in trainFeature.columns if col.startswith('c-')] #cから始まる列名のセット
category_features = ["cp_type", "cp_dose"]
numeric_features = [c for c in trainFeature.columns if c != "sig_id" and c not in category_features]
all_features = category_features + numeric_features

#efficientnet
model_type = "b3"
pretrained_model = f"tf_efficientnet_{model_type}_ns"
#experiment_name = f"deepinsight_efficientnet_v4_{model_type}"
num_workers = 2
gpus = [0]

if model_type == "b0":
    batch_size =  128#128
    infer_batch_size =  128#256
    image_size = 224  # B0
    drop_rate = 0.2  # B0
    resolution = 224
elif model_type == "b3":
    batch_size = 32
    infer_batch_size = 64
    image_size = 300  # B3
    drop_rate = 0.3  # B3
    resolution = 300
elif model_type == "b5":
    batch_size = 8
    infer_batch_size = 16
    image_size = 456  # B5
    drop_rate = 0.4  # B5
    resolution = 456
elif model_type == "b7":
    batch_size = 2
    infer_batch_size = 4
    # image_size = 800  # B7
    image_size = 772  # B7
    drop_rate = 0.5  # B7
    resolution = 772
    
# DeepInsight Transform
perplexity = 5

drop_connect_rate = 0.2
fc_size = 512

# Swap Noise
swap_prob = 0.15
swap_portion = 0.1

SAVEOOF = f"{OUTPUT}/OOF/Efficientnet{model_type}/"

In [6]:
#Seed固定
def seed_everything(seed=42):
    #data取得についてのランダム性固定
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #cudnnによる演算の安定化(評価値の安定)
    torch.backends.cudnn.deterministic = True
    
    #os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(seed=42)

In [7]:
#HyperParameter
param_space = {'hidden_size1': 512, 
               'hidden_size2': 512, 
               'dropOutRate1': 0.20393004966355735, 
               'dropOutRate2': 0.39170486751620137,
               'rankGauss_n_quantiles': 488.0393350201078,
               'leakyReluSlope': 0.01973893854348531,
              }

# Preprocessing

## Category Feature Encoding

In [8]:
def categoryFeatureEnc(trainFeature, testFeature):
    train = trainFeature.copy()
    test = testFeature.copy()
    for df in [train, test]:
        df['cp_type'] = df['cp_type'].map({'ctl_vehicle': 0, 'trt_cp': 1})
        df['cp_dose'] = df['cp_dose'].map({'D1': 0, 'D2': 1})
        df['cp_time'] = df['cp_time'].map({24: 0, 48: 0.5, 72: 1})
    
    return train, test

## Log scaler

In [9]:
# Modified from DeepInsight Transform
# https://github.com/alok-ai-lab/DeepInsight/blob/master/pyDeepInsight/image_transformer.py


class LogScaler:
    """Log normalize and scale data

    Log normalization and scaling procedure as described as norm-2 in the
    DeepInsight paper supplementary information.
    
    Note: The dimensions of input matrix is (N samples, d features)
    """
    def __init__(self):
        self._min0 = None
        self._max = None

    """
    Use this as a preprocessing step in inference mode.
    """
    def fit(self, X, y=None):
        # Min. of training set per feature
        self._min0 = X.min(axis=0)

        # Log normalized X by log(X + _min0 + 1)
        X_norm = np.log(
            X +
            np.repeat(np.abs(self._min0)[np.newaxis, :], X.shape[0], axis=0) +
            1).clip(min=0, max=None)

        # Global max. of training set from X_norm
        self._max = X_norm.max()

    """
    For training set only.
    """
    def fit_transform(self, X, y=None):
        # Min. of training set per feature
        self._min0 = X.min(axis=0)

        # Log normalized X by log(X + _min0 + 1)
        X_norm = np.log(
            X +
            np.repeat(np.abs(self._min0)[np.newaxis, :], X.shape[0], axis=0) +
            1).clip(min=0, max=None)

        # Global max. of training set from X_norm
        self._max = X_norm.max()

        # Normalized again by global max. of training set
        return (X_norm / self._max).clip(0, 1)

    """
    For validation and test set only.
    """
    def transform(self, X, y=None):
        # Adjust min. of each feature of X by _min0
        for i in range(X.shape[1]):
            X[:, i] = X[:, i].clip(min=self._min0[i], max=None)

        # Log normalized X by log(X + _min0 + 1)
        X_norm = np.log(
            X +
            np.repeat(np.abs(self._min0)[np.newaxis, :], X.shape[0], axis=0) +
            1).clip(min=0, max=None)

        # Normalized again by global max. of training set
        return (X_norm / self._max).clip(0, 1)

## DeepInsight Transform

### テーブルデータをCNN用の画像に変換する

In [10]:
# Modified from DeepInsight Transform
# https://github.com/alok-ai-lab/DeepInsight/blob/master/pyDeepInsight/image_transformer.py

#詳細な説明は以下
# https://www.kaggle.com/markpeng/deepinsight-transforming-non-image-data-to-images

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
from scipy.spatial import ConvexHull
from matplotlib import pyplot as plt
import inspect


class DeepInsightTransformer:
    """Transform features to an image matrix using dimensionality reduction

    This class takes in data normalized between 0 and 1 and converts it to a
    CNN compatible 'image' matrix

    """
    def __init__(self,
                 feature_extractor='tsne',
                 perplexity=30,
                 pixels=100,
                 random_state=None,
                 n_jobs=None):
        """Generate an ImageTransformer instance

        Args:
            feature_extractor: string of value ('tsne', 'pca', 'kpca') or a
                class instance with method `fit_transform` that returns a
                2-dimensional array of extracted features.
            pixels: int (square matrix) or tuple of ints (height, width) that
                defines the size of the image matrix.
            random_state: int or RandomState. Determines the random number
                generator, if present, of a string defined feature_extractor.
            n_jobs: The number of parallel jobs to run for a string defined
                feature_extractor.
        """
        self.random_state = random_state
        self.n_jobs = n_jobs

        if isinstance(feature_extractor, str):
            fe = feature_extractor.casefold()
            if fe == 'tsne_exact'.casefold():
                fe = TSNE(n_components=2,
                          metric='cosine',
                          perplexity=perplexity,
                          n_iter=1000,
                          method='exact',
                          random_state=self.random_state,
                          n_jobs=self.n_jobs)
            elif fe == 'tsne'.casefold():
                fe = TSNE(n_components=2,
                          metric='cosine',
                          perplexity=perplexity,
                          n_iter=1000,
                          method='barnes_hut',
                          random_state=self.random_state,
                          n_jobs=self.n_jobs)
            elif fe == 'pca'.casefold():
                fe = PCA(n_components=2, random_state=self.random_state)
            elif fe == 'kpca'.casefold():
                fe = KernelPCA(n_components=2,
                               kernel='rbf',
                               random_state=self.random_state,
                               n_jobs=self.n_jobs)
            else:
                raise ValueError(("Feature extraction method '{}' not accepted"
                                  ).format(feature_extractor))
            self._fe = fe
        elif hasattr(feature_extractor, 'fit_transform') and \
                inspect.ismethod(feature_extractor.fit_transform):
            self._fe = feature_extractor
        else:
            raise TypeError('Parameter feature_extractor is not a '
                            'string nor has method "fit_transform"')

        if isinstance(pixels, int):
            pixels = (pixels, pixels)

        # The resolution of transformed image
        self._pixels = pixels
        self._xrot = None

    def fit(self, X, y=None, plot=False):
        """Train the image transformer from the training set (X)

        Args:
            X: {array-like, sparse matrix} of shape (n_samples, n_features)
            y: Ignored. Present for continuity with scikit-learn
            plot: boolean of whether to produce a scatter plot showing the
                feature reduction, hull points, and minimum bounding rectangle

        Returns:
            self: object
        """
        # Transpose to get (n_features, n_samples)
        X = X.T

        # Perform dimensionality reduction
        x_new = self._fe.fit_transform(X)

        # Get the convex hull for the points
        chvertices = ConvexHull(x_new).vertices
        hull_points = x_new[chvertices]

        # Determine the minimum bounding rectangle
        mbr, mbr_rot = self._minimum_bounding_rectangle(hull_points)

        # Rotate the matrix
        # Save the rotated matrix in case user wants to change the pixel size
        self._xrot = np.dot(mbr_rot, x_new.T).T

        # Determine feature coordinates based on pixel dimension
        self._calculate_coords()

        # plot rotation diagram if requested
        if plot is True:
            # Create subplots
            fig, ax = plt.subplots(1, 1, figsize=(10, 7), squeeze=False)
            ax[0, 0].scatter(x_new[:, 0],
                             x_new[:, 1],
                             cmap=plt.cm.get_cmap("jet", 10),
                             marker="x",
                             alpha=1.0)
            ax[0, 0].fill(x_new[chvertices, 0],
                          x_new[chvertices, 1],
                          edgecolor='r',
                          fill=False)
            ax[0, 0].fill(mbr[:, 0], mbr[:, 1], edgecolor='g', fill=False)
            plt.gca().set_aspect('equal', adjustable='box')
            plt.show()
        return self

    @property
    def pixels(self):
        """The image matrix dimensions

        Returns:
            tuple: the image matrix dimensions (height, width)

        """
        return self._pixels

    @pixels.setter
    def pixels(self, pixels):
        """Set the image matrix dimension

        Args:
            pixels: int or tuple with the dimensions (height, width)
            of the image matrix

        """
        if isinstance(pixels, int):
            pixels = (pixels, pixels)
        self._pixels = pixels
        # recalculate coordinates if already fit
        if hasattr(self, '_coords'):
            self._calculate_coords()

    def _calculate_coords(self):
        """Calculate the matrix coordinates of each feature based on the
        pixel dimensions.
        """
        ax0_coord = np.digitize(self._xrot[:, 0],
                                bins=np.linspace(min(self._xrot[:, 0]),
                                                 max(self._xrot[:, 0]),
                                                 self._pixels[0])) - 1
        ax1_coord = np.digitize(self._xrot[:, 1],
                                bins=np.linspace(min(self._xrot[:, 1]),
                                                 max(self._xrot[:, 1]),
                                                 self._pixels[1])) - 1
        self._coords = np.stack((ax0_coord, ax1_coord))

    def transform(self, X, empty_value=0):
        """Transform the input matrix into image matrices

        Args:
            X: {array-like, sparse matrix} of shape (n_samples, n_features)
                where n_features matches the training set.
            empty_value: numeric value to fill elements where no features are
                mapped. Default = 0 (although it was 1 in the paper).

        Returns:
            A list of n_samples numpy matrices of dimensions set by
            the pixel parameter
        """

        # Group by location (x1, y1) of each feature
        # Tranpose to get (n_features, n_samples)
        img_coords = pd.DataFrame(np.vstack(
            (self._coords, X.clip(0, 1))).T).groupby(
                [0, 1],  # (x1, y1)
                as_index=False).mean()

        img_matrices = []
        blank_mat = np.zeros(self._pixels)
        if empty_value != 0:
            blank_mat[:] = empty_value
        for z in range(2, img_coords.shape[1]):
            img_matrix = blank_mat.copy()
            img_matrix[img_coords[0].astype(int),
                       img_coords[1].astype(int)] = img_coords[z]
            img_matrices.append(img_matrix)

        return img_matrices

    def fit_transform(self, X, empty_value=0):
        """Train the image transformer from the training set (X) and return
        the transformed data.

        Args:
            X: {array-like, sparse matrix} of shape (n_samples, n_features)
            empty_value: numeric value to fill elements where no features are
                mapped. Default = 0 (although it was 1 in the paper).

        Returns:
            A list of n_samples numpy matrices of dimensions set by
            the pixel parameter
        """
        self.fit(X)
        return self.transform(X, empty_value=empty_value)

    def feature_density_matrix(self):
        """Generate image matrix with feature counts per pixel

        Returns:
            img_matrix (ndarray): matrix with feature counts per pixel
        """
        fdmat = np.zeros(self._pixels)
        # Group by location (x1, y1) of each feature
        # Tranpose to get (n_features, n_samples)
        coord_cnt = (
            pd.DataFrame(self._coords.T).assign(count=1).groupby(
                [0, 1],  # (x1, y1)
                as_index=False).count())
        fdmat[coord_cnt[0].astype(int),
              coord_cnt[1].astype(int)] = coord_cnt['count']
        return fdmat

    @staticmethod
    def _minimum_bounding_rectangle(hull_points):
        """Find the smallest bounding rectangle for a set of points.

        Modified from JesseBuesking at https://stackoverflow.com/a/33619018
        Returns a set of points representing the corners of the bounding box.

        Args:
            hull_points : an nx2 matrix of hull coordinates

        Returns:
            (tuple): tuple containing
                coords (ndarray): coordinates of the corners of the rectangle
                rotmat (ndarray): rotation matrix to align edges of rectangle
                    to x and y
        """

        pi2 = np.pi / 2.

        # Calculate edge angles
        edges = hull_points[1:] - hull_points[:-1]
        angles = np.arctan2(edges[:, 1], edges[:, 0])
        angles = np.abs(np.mod(angles, pi2))
        angles = np.unique(angles)

        # Find rotation matrices
        rotations = np.vstack([
            np.cos(angles),
            np.cos(angles - pi2),
            np.cos(angles + pi2),
            np.cos(angles)
        ]).T
        rotations = rotations.reshape((-1, 2, 2))

        # Apply rotations to the hull
        rot_points = np.dot(rotations, hull_points.T)

        # Find the bounding points
        min_x = np.nanmin(rot_points[:, 0], axis=1)
        max_x = np.nanmax(rot_points[:, 0], axis=1)
        min_y = np.nanmin(rot_points[:, 1], axis=1)
        max_y = np.nanmax(rot_points[:, 1], axis=1)

        # Find the box with the best area
        areas = (max_x - min_x) * (max_y - min_y)
        best_idx = np.argmin(areas)

        # Return the best box
        x1 = max_x[best_idx]
        x2 = min_x[best_idx]
        y1 = max_y[best_idx]
        y2 = min_y[best_idx]
        rotmat = rotations[best_idx]

        # Generate coordinates
        coords = np.zeros((4, 2))
        coords[0] = np.dot([x1, y2], rotmat)
        coords[1] = np.dot([x2, y2], rotmat)
        coords[2] = np.dot([x2, y1], rotmat)
        coords[3] = np.dot([x1, y1], rotmat)

        return coords, rotmat

## Collecting

__train,testにターゲット値も連結__

In [11]:
def Collecting(trainFeature, testFeature, trainTargetScored):
    #Pkey(sig_id)でfeatureとtargetを内部結合。
    train = trainFeature.merge(trainTargetScored, on='sig_id')
    test = testFeature.merge(sample_submission, on='sig_id')
    
    return train, test

## Preprocessing Summary

In [12]:
def preprocessing(param, trainFeature, testFeature, trainTargetScored):
    
    train, test = categoryFeatureEnc(trainFeature, testFeature)
    
    train, test = Collecting(train, test, trainTargetScored)
    
    return train, test

## Work

## Visualization

In [13]:
%%time
trainVsl, testVsl = preprocessing(param_space, trainFeature, testFeature, trainTargetScored)

CPU times: user 53.4 ms, sys: 44 ms, total: 97.4 ms
Wall time: 96.7 ms


In [14]:
trainVsl

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,1,0.0,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,1,1.0,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,1,0.5,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,1,0.5,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,1,1.0,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,1,0.0,1,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0,0,0,0,0,0,0,0,0,0
23810,id_fffb70c0c,1,0.0,1,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,0,0,0,0,0,0,0,0,0,0
23811,id_fffc1c3f4,0,0.5,1,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,0,0,0,0,0,0,0,0,0,0
23812,id_fffcb9e7c,1,0.0,0,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,0,0,0,0,0,0,0,0,0,0


In [15]:
testVsl

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,1,0.0,0,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,id_001897cda,1,1.0,0,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,id_002429b5b,0,0.0,0,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3,id_00276f245,1,0.0,1,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,id_0027f1083,1,0.5,0,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,1,0.0,0,0.4571,-0.5743,3.3930,-0.6202,0.8557,1.6240,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3978,id_ff925dd0d,1,0.0,0,-0.5885,-0.2548,2.5850,0.3456,0.4401,0.3107,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3979,id_ffb710450,1,1.0,0,-0.3985,-0.1554,0.2677,-0.6813,0.0152,0.4791,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3980,id_ffbb869f2,1,0.5,1,-1.0960,-1.7750,-0.3977,1.0160,-1.3350,-0.2207,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [16]:
trainFeature

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,id_fffc1c3f4,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


In [17]:
testFeature

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,trt_cp,24,D1,0.4571,-0.5743,3.3930,-0.6202,0.8557,1.6240,...,-1.1790,-0.6422,-0.4367,0.0159,-0.6539,-0.4791,-1.2680,-1.1280,-0.4167,-0.6600
3978,id_ff925dd0d,trt_cp,24,D1,-0.5885,-0.2548,2.5850,0.3456,0.4401,0.3107,...,0.0210,0.5780,-0.5888,0.8057,0.9312,1.2730,0.2614,-0.2790,-0.0131,-0.0934
3979,id_ffb710450,trt_cp,72,D1,-0.3985,-0.1554,0.2677,-0.6813,0.0152,0.4791,...,0.4418,0.9153,-0.1862,0.4049,0.9568,0.4666,0.0461,0.5888,-0.4205,-0.1504
3980,id_ffbb869f2,trt_cp,48,D2,-1.0960,-1.7750,-0.3977,1.0160,-1.3350,-0.2207,...,0.3079,-0.4473,-0.8192,0.7785,0.3133,0.1286,-0.2618,0.5074,0.7430,-0.0484


# Fitting

## Config about Fitting

In [18]:
#configは辞書化しておく。
def Config_about_Fitting(train, test, target, folds):
    confFitting = {}
    
    #Fitするときに"y"として使う列の列名配列
    confFitting["target_cols"] = target.drop('sig_id', axis=1).columns.values.tolist()
    #Fitするときに"X"として使う列の列名配列
    #kfold, id等はここで削除。
    feature_cols = [c for c in folds.columns if c not in confFitting["target_cols"]]
    confFitting["feature_cols"] = [c for c in feature_cols if c not in ['kfold','sig_id']]
    #特徴量、ターゲットのサイズ
    confFitting["num_features"]=len(confFitting["feature_cols"])
    confFitting["num_targets"]=len(confFitting["target_cols"])
    
    return confFitting

## fitTransformer

In [19]:
def fitPreprocessingModel(param, train, test, valid, fold, seed):
    
    #LOG SCALER
    train_all_features = train[all_features].copy().values
    valid_all_features = valid[all_features].copy().values
    test_all_features = test[all_features].copy().values
    
    all_scaler = LogScaler()
    train_all_features = all_scaler.fit_transform(train_all_features)
    test_all_features = all_scaler.transform(test_all_features)
    valid_all_features = all_scaler.transform(valid_all_features)
    
    train[all_features] = train_all_features
    test[all_features] = test_all_features
    valid[all_features] = valid_all_features
    
    dump(all_scaler, open(f"{SAVELOGSCALE}/seed{seed}_fold{fold}_LogScaleTransformer.pkl", 'wb'), pickle.HIGHEST_PROTOCOL)
    
    #DeepInsight
    all_it = DeepInsightTransformer(feature_extractor='tsne_exact',
                                    pixels=resolution,
                                    perplexity=5,
                                    random_state=1120,
                                    n_jobs=-1)
    all_it.fit(train_all_features, plot=False)
    
    dump(all_it, open(f"{SAVEDEEPINSIGHT}/seed{seed}_fold{fold}_DeepInsightTransformer.pkl", 'wb'), pickle.HIGHEST_PROTOCOL)
    
    return all_scaler, all_it, train, test, valid

In [20]:
def PreprocessingLoadTransform(param, test, fold, seed):
    test_all_features = test[all_features].copy().values
    
    #LOG SCALER
    all_scaler = load(open(f"{SAVELOGSCALE}/seed{seed}_fold{fold}_LogScaleTransformer.pkl", 'rb'))
    test_all_features = all_scaler.transform(test_all_features)
    
    #DeepInsight
    all_it = load(open(f"{SAVEDEEPINSIGHT}/seed{seed}_fold{fold}_DeepInsightTransformer.pkl", 'rb'))
    
    return all_scaler, all_it, test

## Dataset Classes

In [21]:
class MoAImageSwapDataset(torch.utils.data.Dataset):
    def __init__(self,
                 features,
                 labels,
                 transformer,
                 swap_prob=0.15,
                 swap_portion=0.1):
        self.features = features
        self.labels = labels
        self.transformer = transformer
        self.swap_prob = swap_prob
        self.swap_portion = swap_portion

    def __getitem__(self, index):
        normalized = self.features[index, :]

        # Swap row features randomly
        normalized = self.add_swap_noise(index, normalized)

        normalized = np.expand_dims(normalized, axis=0)

        # Note: we are setting empty_value=1 to follow the setup in the paper
        image = self.transformer.transform(normalized, empty_value=1)[0]

        # Resize to target size
        gene_cht = cv2.resize(image, (image_size, image_size),
                              interpolation=cv2.INTER_CUBIC)

        # Convert to 3 channels
        image = np.repeat(gene_cht[np.newaxis, :, :], 3, axis=0)

        return {"x": image, "y": self.labels[index, :]}

    def add_swap_noise(self, index, X):
        if np.random.rand() < self.swap_prob:
            swap_index = np.random.randint(self.features.shape[0], size=1)[0]
            # Select only gene expression and cell viability features
            swap_features = np.random.choice(
                np.array(range(3, self.features.shape[1])),
                size=int(self.features.shape[1] * self.swap_portion),
                replace=False)
            X[swap_features] = self.features[swap_index, swap_features]

        return X

    def __len__(self):
        return self.features.shape[0]
    
class MoAImageDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels, transformer):
        self.features = features
        self.labels = labels
        self.transformer = transformer

    def __getitem__(self, index):
        normalized = self.features[index, :]
        normalized = np.expand_dims(normalized, axis=0)

        # Note: we are setting empty_value=1 to follow the setup in the paper
        image = self.transformer.transform(normalized, empty_value=1)[0]

        # Resize to target size
        gene_cht = cv2.resize(image, (image_size, image_size),
                              interpolation=cv2.INTER_CUBIC)

        # Convert to 3 channels
        image = np.repeat(gene_cht[np.newaxis, :, :], 3, axis=0)

        return {"x": image, "y": self.labels[index, :]}

    def __len__(self):
        return self.features.shape[0]


class TestDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels, transformer):
        self.features = features
        self.labels = labels
        self.transformer = transformer

    def __getitem__(self, index):
        normalized = self.features[index, :]
        normalized = np.expand_dims(normalized, axis=0)

        # Note: we are setting empty_value=1 to follow the setup in the paper
        image = self.transformer.transform(normalized, empty_value=1)[0]

        # Resize to target size
        gene_cht = cv2.resize(image, (image_size, image_size),
                              interpolation=cv2.INTER_CUBIC)

        # Convert to 3 channels
        image = np.repeat(gene_cht[np.newaxis, :, :], 3, axis=0)

        return {"x": image, "y": -1}

    def __len__(self):
        return self.features.shape[0]

## weight initialization

In [22]:
# Reference: https://github.com/rwightman/gen-efficientnet-pytorch/blob/master/geffnet/efficientnet_builder.py#L672
def initialize_weight_goog(m, n='', fix_group_fanout=True):
    # weight init as per Tensorflow Official impl
    # https://github.com/tensorflow/tpu/blob/master/models/official/mnasnet/mnasnet_model.py
    if isinstance(m, nn.Conv2d):
        fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        if fix_group_fanout:
            fan_out //= m.groups
        m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
        if m.bias is not None:
            m.bias.data.zero_()
    elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1.0)
        m.bias.data.zero_()
    elif isinstance(m, nn.Linear):
        fan_out = m.weight.size(0)  # fan-out
        fan_in = 0
        if 'routing_fn' in n:
            fan_in = m.weight.size(1)
        init_range = 1.0 / math.sqrt(fan_in + fan_out)
        m.weight.data.uniform_(-init_range, init_range)
        m.bias.data.zero_()


def initialize_weight_default(m, n=''):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
    elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1.0)
        m.bias.data.zero_()
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight,
                                 mode='fan_in',
                                 nonlinearity='linear')

## Loss, Metric

In [23]:
#loss
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self):
        super(LabelSmoothingCrossEntropy, self).__init__()
    def forward(self, x, target, smoothing=0.001):
        confidence = 1. - smoothing
        logprobs = F.log_softmax(x, dim=-1)
        bcs_loss = nn.BCEWithLogitsLoss()(x, target)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * bcs_loss + smoothing * smooth_loss
        return loss.mean()

In [24]:
#metric
#nn.BCEWithLogitsLoss()

## Model architect

In [26]:
class MoAEfficientNet(pl.LightningModule):
    def __init__(
            self,
            pretrained_model_name,
            training_set=(None, None),  # tuple
            valid_set=(None, None),  # tuple
            test_set=None,
            transformer=None,
            num_classes=206,
            in_chans=3,
            drop_rate=0.,
            drop_connect_rate=0.,
            fc_size=512,
            learning_rate=1e-3,
            weight_init='goog'):
        super(MoAEfficientNet, self).__init__()

        self.train_data, self.train_labels = training_set
        self.valid_data, self.valid_labels = valid_set
        self.test_data = test_set
        self.transformer = transformer

        self.backbone = getattr(geffnet, pretrained_model)(
            pretrained=True,
            in_chans=in_chans,
            drop_rate=drop_rate,
            drop_connect_rate=drop_connect_rate,
            weight_init=weight_init)

        self.backbone.classifier = nn.Sequential(
            nn.Linear(self.backbone.classifier.in_features, fc_size,
                      bias=True), nn.ELU(),
            nn.Linear(fc_size, num_classes, bias=True))

        if self.training:
            for m in self.backbone.classifier.modules():
                initialize_weight_goog(m)

        # Save passed hyperparameters
        self.save_hyperparameters("pretrained_model_name", "num_classes",
                                  "in_chans", "drop_rate", "drop_connect_rate",
                                  "weight_init", "fc_size", "learning_rate")

    def forward(self, x):
        return self.backbone(x)

    def training_step(self, batch, batch_idx):
        x = batch["x"]
        y = batch["y"]
        x = x.float()
        y = y.type_as(x)
        logits = self(x)

        loss = F.binary_cross_entropy_with_logits(logits, y, reduction="mean")
        #loss = LabelSmoothingCrossEntropy()

        self.log('train_loss',
                 loss,
                 on_step=True,
                 on_epoch=True,
                 prog_bar=True,
                 logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x = batch["x"]
        y = batch["y"]
        x = x.float()
        y = y.type_as(x)
        logits = self(x)

        val_loss = F.binary_cross_entropy_with_logits(logits,
                                                      y,
                                                      reduction="mean")

        self.log('val_loss',
                 val_loss,
                 on_step=True,
                 on_epoch=True,
                 prog_bar=True,
                 logger=True)

        return val_loss

    def test_step(self, batch, batch_idx):
        x = batch["x"]
        y = batch["y"]
        x = x.float()
        y = y.type_as(x)
        logits = self(x)
        return {"pred_logits": logits}

    def test_epoch_end(self, output_results):
        all_outputs = torch.cat([out["pred_logits"] for out in output_results],
                                dim=0)
        print("Logits:", all_outputs)
        pred_probs = F.sigmoid(all_outputs).detach().cpu().numpy()
        print("Predictions: ", pred_probs)
        return {"pred_probs": pred_probs}

    def setup(self, stage=None):
        #         self.train_dataset = MoAImageDataset(self.train_data,
        #                                              self.train_labels,
        #                                              self.transformer)
        self.train_dataset = MoAImageSwapDataset(self.train_data,
                                                 self.train_labels,
                                                 self.transformer,
                                                 swap_prob=swap_prob,
                                                 swap_portion=swap_portion)

        self.val_dataset = MoAImageDataset(self.valid_data, self.valid_labels,
                                           self.transformer)

        self.test_dataset = TestDataset(self.test_data, None, self.transformer)

    def train_dataloader(self):
        train_dataloader = DataLoader(self.train_dataset,
                                      batch_size=batch_size,
                                      shuffle=True,
                                      num_workers=num_workers,
                                      pin_memory=True,
                                      drop_last=False)
        print(f"Train iterations: {len(train_dataloader)}")
        return train_dataloader

    def val_dataloader(self):
        val_dataloader = DataLoader(self.val_dataset,
                                    batch_size=infer_batch_size,
                                    shuffle=False,
                                    num_workers=num_workers,
                                    pin_memory=True,
                                    drop_last=False)
        print(f"Validate iterations: {len(val_dataloader)}")
        return val_dataloader

    def test_dataloader(self):
        test_dataloader = DataLoader(self.test_dataset,
                                     batch_size=infer_batch_size,
                                     shuffle=False,
                                     num_workers=num_workers,
                                     pin_memory=True,
                                     drop_last=False)
        print(f"Test iterations: {len(test_dataloader)}")
        return test_dataloader

    def configure_optimizers(self):
        print(f"Initial Learning Rate: {self.hparams.learning_rate:.6f}")
        optimizer = optim.Adam(self.parameters(),
                               lr=self.hparams.learning_rate,
                               weight_decay=1e-5)

        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                         T_max=20,
                                                         eta_min=0,
                                                         last_epoch=-1)

        return [optimizer], [scheduler]

# Run

## HyperParameter

In [27]:
# HyperParameters
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

## CV folds

In [28]:
def CV_folds(train, target):
    folds = train.copy()
    
    mskf = MultilabelStratifiedKFold(n_splits=NFOLDS)
    
    for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
        folds.loc[v_idx, 'kfold'] = int(f)
    
    folds['kfold'] = folds['kfold'].astype(int)
    
    return folds

In [29]:
def CV_folds_drug_id(train, target):
    ###drug_idを考慮####
    
    targets = target.columns[1:]
    
    # foldsにdrug_id付与
    folds = train.copy()
    folds = folds.merge(drug, on='sig_id', how='left') 
    
    # LOCATE DRUGS
    vc = folds.drug_id.value_counts()
    vc1 = vc.loc[vc<=18].index.sort_values()
    vc2 = vc.loc[vc>18].index.sort_values()
    
    # STRATIFY DRUGS 18X OR LESS
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
              random_state=42)
    tmp = folds.groupby('drug_id')[targets].mean().loc[vc1]
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.index[idxV].values}
        dct1.update(dd)
    
    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
              random_state=42)
    tmp = folds.loc[folds.drug_id.isin(vc2)].reset_index(drop=True)
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.sig_id[idxV].values}
        dct2.update(dd)
    
    # ASSIGN NFOLDS
    folds['kfold'] = folds.drug_id.map(dct1)
    folds.loc[folds.kfold.isna(),'kfold'] =\
        folds.loc[folds.kfold.isna(),'sig_id'].map(dct2)
    folds.kfold = folds.kfold.astype('int8')
    
    folds = folds.drop('drug_id', axis=1)
    
    return folds

In [30]:
%%time
#Preprocessing Data
trainVsl, testVsl = preprocessing(param_space, trainFeature, testFeature, trainTargetScored)
#CV folds
foldsVsl = CV_folds_drug_id(trainVsl, trainTargetScored)

foldsVsl.head(5)

CPU times: user 474 ms, sys: 137 ms, total: 611 ms
Wall time: 609 ms


,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold
0,id_000644bb2,1,0.0,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0,0,0,0,0,0,0,0,0,4
1,id_000779bfc,1,1.0,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,0,0,0,0,0,0,0,0,0,2
2,id_000a6266a,1,0.5,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,0,0,0,0,0,0,0,0,0,4
3,id_0015fd391,1,0.5,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,1,1.0,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0,0,0,0,0,0,0,0,0,1


## Single Fold Running

In [31]:
def run_training(confFitting, Tester, fold, seed, param,
                 folds, train, test, target):
    
    seed_everything(seed)
    
    train = folds
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[confFitting["feature_cols"]], train_df[confFitting["target_cols"]].values
    x_valid, y_valid =  valid_df[confFitting["feature_cols"]], valid_df[confFitting["target_cols"]].values
    x_test = test[confFitting["feature_cols"]]
    
    #データセットをイメージ化するトランスフォーマー。
    #ここでLogScaleも実施。
    all_scaler, all_it, x_train, x_test, x_valid = fitPreprocessingModel(param_space, x_train, x_test, x_valid, fold, seed)
    x_train = x_train.values
    x_valid = x_valid.values
    x_test = x_test.values
    
    #model class 定義
    model = MoAEfficientNet(
        pretrained_model_name=pretrained_model,
        training_set=(x_train, y_train),  # tuple
        valid_set=(x_valid, y_valid),  # tuple
        test_set=np.concatenate([x_valid, x_test], 0), #予測用のデータセット
        transformer=all_it,
        drop_rate=drop_rate,
        drop_connect_rate=drop_connect_rate,
        fc_size=fc_size,
        weight_init='goog')
    
    #データセット定義(データセットをイメージ化)
    model.setup()
    
    #学習クラス
    trainer = Trainer(
        default_root_dir=SAVEMODEL,
        max_epochs=EPOCHS,
        logger=False,
        gpus=gpus,
        distributed_backend="dp",  # multiple-gpus, 1 machine
        precision=16)
    
    #学習
    trainer.fit(model)
    
    #save model
    trainer.save_checkpoint(f"{SAVEMODEL}/model{model_type}_SEED{seed}_FOLD{fold}.ckpt")
    
    #予測
    output = trainer.test(model, verbose=False)[0]
    allPred = output["pred_probs"]
    
    #out of fold
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    oof[val_idx] = allPred[0:x_valid.shape[0]]
    
    #Predictions
    predictions = allPred[x_valid.shape[0]:]
    
    
    return oof, predictions


## K-Fold Running

In [32]:
def run_k_fold(Tester, NFOLDS, seed, param,
               folds, train, test, target, confFitting):
    oof = np.zeros((len(train), confFitting["num_targets"]))
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    for fold in range(NFOLDS):
        if Tester:
            print('=' * 20, 'Fold', fold, '=' * 20)
        oof_, pred_ = run_training(confFitting, Tester, fold, seed, param,
                                   folds, train, test, target)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

## CV Evaluation

In [33]:
 def CV_Evaluation(confFitting, oof, train, target):
    #CV score : OOFの評価結果。
    #OOF(学習モデルによるtrain dataの予測)
    train[confFitting["target_cols"]] = oof
    #target(予測結果)：ここで処理「cp_type = ctl_vehicleのレコードを削除」で抜けたところに0を入れている。
    valid_results = trainTargetScored.drop(columns=confFitting["target_cols"]).merge(train[['sig_id']+confFitting["target_cols"]], on='sig_id', how='left').fillna(0)
    
    y_true = trainTargetScored[confFitting["target_cols"]].values
    y_pred = valid_results[confFitting["target_cols"]].values
    
    score = 0
    for i in range(confFitting["num_targets"]):
        score_ = log_loss(y_true[:, i], y_pred[:, i]) #問題の評価指標によって変わる。
        score += score_ / target.shape[1]
        
    print("CV log_loss: ", score)
    
    #OOF save
    np.save(SAVEOOF + 'oof', y_pred)
    
    return score

## Postprocessing

In [34]:
# 特になし

## Submit

In [35]:
def Submit(confFitting, predictions, test):
    test[confFitting["target_cols"]] = predictions
    sub = sample_submission.drop(columns=confFitting["target_cols"]).merge(test[['sig_id']+confFitting["target_cols"]], on='sig_id', how='left').fillna(0)
    sub.to_csv(f'{SUBMIT}submission.csv', index=False)

    print("sub.shape" + str(sub.shape))
    
    return

# Execute

In [36]:
def Exec(param):
    
    #Tester(True/False)
    Tester = True
    
    #Preprocessing Data
    train, test = preprocessing(param, trainFeature, testFeature, trainTargetScored)
    
    #CV folds
    folds = CV_folds_drug_id(train, trainTargetScored)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, test, trainTargetScored, folds)
    
    # Averaging on multiple SEEDS
    SEED = [42]
    oof = np.zeros((len(train), confFitting["num_targets"]))
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        oof_, predictions_ = run_k_fold(Tester, NFOLDS, seed, param,
                                       folds, train, test, trainTargetScored, confFitting)
        oof += oof_ / len(SEED)
        predictions += predictions_ / len(SEED)
    
    #CV 評価
    score = CV_Evaluation(confFitting, oof, train, trainTargetScored)
    
    # 課題提出
    Submit(confFitting, predictions, test)
    
    return score


In [36]:
%%time
#b3
score= Exec(param_space)
print("score: " + str(score))

~~~~~~~~~~~~~~~~~~~~ SEED 42 ~~~~~~~~~~~~~~~~~~~~
==================== Fold 0 ====================


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.

  | Name     | Type            | Params
---------------------------------------------
0 | backbone | GenEfficientNet | 11 M  


Initial Learning Rate: 0.001000
Validate iterations: 76
Train iterations: 594                                                 
Epoch 0:  89%|████████▊ | 594/670 [03:37<00:27,  2.74it/s, loss=0.020, val_loss_step=0.698, train_loss_step=0.0192]
Validating: 0it [00:00, ?it/s]
Epoch 0:  99%|█████████▉| 662/670 [03:48<00:02,  2.89it/s, loss=0.020, val_loss_step=0.698, train_loss_step=0.0192]


Epoch 1:  89%|████████▊ | 594/670 [03:37<00:27,  2.73it/s, loss=0.019, val_loss_step=0.0209, train_loss_step=0.0172, val_loss_epoch=0.0206, train_loss_epoch=0.0263]
Validating: 0it [00:00, ?it/s]
Epoch 1:  95%|█████████▍| 636/670 [03:44<00:12,  2.83it/s, loss=0.019, val_loss_step=0.0209, train_loss_step=0.0172, val_loss_epoch=0.0206, train_loss_epoch=0.0263]


Epoch 2:  89%|████████▊ | 594/670 [03:37<00:27,  2.73it/s, loss=0.018, val_loss_step=0.0209, train_loss_step=0.0184, val_loss_epoch=0.0205, train_loss_epoch=0.0194]
Validating: 0it [00:00, ?it/s]
Epoch 2:  91%|█████████ | 608/670 [03:39<00:22,  2.77it/s, loss=0.018, val_loss_step=0.0209, train_loss_step=0.0184, val_loss_epoch=0.0205, train_loss_epoch=0.0194]


...

Epoch 22:  99%|█████████▊| 661/670 [03:44<00:03,  2.94it/s, loss=0.016, val_loss_step=0.0168, train_loss_step=0.0123, val_loss_epoch=0.0166, train_loss_epoch=0.0157]


Epoch 23:  89%|████████▉ | 595/670 [03:33<00:26,  2.79it/s, loss=0.016, val_loss_step=0.0177, train_loss_step=0.0182, val_loss_epoch=0.0169, train_loss_epoch=0.0158] 
Validating: 0it [00:00, ?it/s]
Epoch 23:  95%|█████████▍| 634/670 [03:39<00:12,  2.88it/s, loss=0.016, val_loss_step=0.0177, train_loss_step=0.0182, val_loss_epoch=0.0169, train_loss_epoch=0.0158]


Epoch 24:  89%|████████▉ | 595/670 [03:33<00:26,  2.79it/s, loss=0.016, val_loss_step=0.019, train_loss_step=0.014, val_loss_epoch=0.0184, train_loss_epoch=0.0158]  
Validating: 0it [00:00, ?it/s]
Epoch 24:  91%|█████████ | 607/670 [03:35<00:22,  2.82it/s, loss=0.016, val_loss_step=0.019, train_loss_step=0.014, val_loss_epoch=0.0184, train_loss_epoch=0.0158]


Epoch 24:  98%|█████████▊| 656/670 [03:43<00:04,  2.93it/s, loss=0.016, val_loss_step=0.019, train_loss_step=0.014, val_loss_epoch=0.0184, train_loss_epoch=0.0158]


Epoch 24: 100%|██████████| 670/670 [03:46<00:00,  2.96it/s, loss=0.016, val_loss_step=0.0171, train_loss_step=0.014, val_loss_epoch=0.0181, train_loss_epoch=0.0158]
Test iterations: 138
Testing: 100%|██████████| 138/138 [00:20<00:00,  6.79it/s]Logits: tensor([[ -5.9414,  -7.1641,  -7.6289,  ...,  -7.3320,  -6.5938,  -6.4961],
        [ -6.6172,  -6.9141,  -7.5391,  ...,  -7.9336,  -6.8945,  -7.4727],
        [ -9.0000, -10.1172, -10.3516,  ..., -10.3672, -10.9453, -10.9297],
        ...,
        [ -5.7461,  -6.9453,  -7.7266,  ...,  -7.4570,  -6.6523,  -6.5781],
        [ -5.8438,  -7.0625,  -7.5156,  ...,  -7.2344,  -6.4023,  -6.3359],
        [ -6.0000,  -7.1602,  -7.5898,  ...,  -7.3711,  -6.5469,  -6.5352]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[2.621e-03 7.734e-04 4.859e-04 ... 6.537e-04 1.367e-03 1.507e-03]
 [1.335e-03 9.928e-04 5.317e-04 ... 3.583e-04 1.012e-03 5.679e-04]
 [1.234e-04 4.035e-05 3.195e-05 ... 3.147e-05 1.764e-05 1.794e-05]
 ...
 [3.185e-03 9

In [36]:
%%time
#b0
score= Exec(param_space)
print("score: " + str(score))

~~~~~~~~~~~~~~~~~~~~ SEED 42 ~~~~~~~~~~~~~~~~~~~~
==================== Fold 0 ====================


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.

  | Name     | Type            | Params
---------------------------------------------
0 | backbone | GenEfficientNet | 4 M   


Initial Learning Rate: 0.001000
Validate iterations: 38
Train iterations: 149                                                 
Epoch 0:  80%|███████▉  | 149/187 [00:59<00:15,  2.50it/s, loss=0.020, val_loss_step=0.688, train_loss_step=0.0206]
Validating: 0it [00:00, ?it/s]
Epoch 1:  80%|███████▉  | 149/187 [00:59<00:15,  2.49it/s, loss=0.019, val_loss_step=0.0205, train_loss_step=0.0172, val_loss_epoch=0.0226, train_loss_epoch=0.0373]
Validating: 0it [00:00, ?it/s]
Epoch 2:  80%|███████▉  | 149/187 [00:59<00:15,  2.48it/s, loss=0.018, val_loss_step=0.0455, train_loss_step=0.0198, val_loss_epoch=0.0485, train_loss_epoch=0.0191]
Validating: 0it [00:00, ?it/s]
Epoch 2:  82%|████████▏ | 154/187 [01:01<00:13,  2.52it/s, loss=0.018, val_loss_step=0.0455, train_loss_step=0.0198, val_loss_epoch=0.0485, train_loss_epoch=0.0191]


Epoch 3:  80%|███████▉  | 149/187 [00:59<00:15,  2.49it/s, loss=0.017, val_loss_step=0.0336, train_loss_step=0.0192, val_loss_epoch=0.0354, train_loss_epoch=0.018] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  80%|███████▉  | 149/187 [01:00<00:15,  2.48it/s, loss=0.017, val_loss_step=0.0436, train_loss_step=0.017, val_loss_epoch=0.0465, train_loss_epoch=0.0172] 
Validating: 0it [00:00, ?it/s]
Validating:  42%|████▏     | 16/38 [00:03<00:03,  5.66it/s]


Epoch 5:  80%|███████▉  | 149/187 [00:59<00:15,  2.49it/s, loss=0.016, val_loss_step=0.0151, train_loss_step=0.0192, val_loss_epoch=0.0184, train_loss_epoch=0.0169]
Validating: 0it [00:00, ?it/s]
Epoch 6:  80%|███████▉  | 149/187 [00:59<00:15,  2.49it/s, loss=0.016, val_loss_step=0.0985, train_loss_step=0.0204, val_loss_epoch=0.105, train_loss_epoch=0.0167]
Validating: 0it [00:00, ?it/s]
Epoch 6:  89%|████████▉ | 166/187 [01:03<00:08,  2.62it/s, loss=0.016, val_loss_step=0.0985, train_loss_step=0.0204, val_loss_epoch=0.105, train_loss_epoch=0.0167]


...

Epoch 16:  80%|███████▉  | 149/187 [01:00<00:15,  2.46it/s, loss=0.015, val_loss_step=0.0239, train_loss_step=0.0167, val_loss_epoch=0.0254, train_loss_epoch=0.0144]
Validating: 0it [00:00, ?it/s]
Epoch 17:  80%|███████▉  | 149/187 [01:00<00:15,  2.47it/s, loss=0.014, val_loss_step=0.0178, train_loss_step=0.0178, val_loss_epoch=0.0173, train_loss_epoch=0.0142]
Validating: 0it [00:00, ?it/s]
Validating:  79%|███████▉  | 30/38 [00:05<00:01,  6.07it/s]


Epoch 18:  80%|███████▉  | 149/187 [01:01<00:15,  2.43it/s, loss=0.014, val_loss_step=0.0171, train_loss_step=0.0147, val_loss_epoch=0.0168, train_loss_epoch=0.014] 
Validating: 0it [00:00, ?it/s]
Epoch 19:  80%|███████▉  | 149/187 [01:00<00:15,  2.46it/s, loss=0.014, val_loss_step=0.0174, train_loss_step=0.0115, val_loss_epoch=0.0166, train_loss_epoch=0.0138]
Validating: 0it [00:00, ?it/s]
Validating:  68%|██████▊   | 26/38 [00:05<00:02,  5.63it/s]


Epoch 20:  80%|███████▉  | 149/187 [01:00<00:15,  2.46it/s, loss=0.014, val_loss_step=0.0175, train_loss_step=0.0183, val_loss_epoch=0.0166, train_loss_epoch=0.0137]
Validating: 0it [00:00, ?it/s]
Epoch 21:  80%|███████▉  | 149/187 [01:00<00:15,  2.46it/s, loss=0.013, val_loss_step=0.0175, train_loss_step=0.0127, val_loss_epoch=0.0166, train_loss_epoch=0.0137]
Validating: 0it [00:00, ?it/s]
Epoch 21:  93%|█████████▎| 174/187 [01:05<00:04,  2.66it/s, loss=0.013, val_loss_step=0.0175, train_loss_step=0.0127, val_loss_epoch=0.0166, train_loss_epoch=0.0137]


Epoch 22:  80%|███████▉  | 149/187 [01:00<00:15,  2.46it/s, loss=0.014, val_loss_step=0.0175, train_loss_step=0.0145, val_loss_epoch=0.0166, train_loss_epoch=0.0137]
Validating: 0it [00:00, ?it/s]
Epoch 23:  80%|███████▉  | 149/187 [01:00<00:15,  2.46it/s, loss=0.014, val_loss_step=0.0175, train_loss_step=0.0159, val_loss_epoch=0.0166, train_loss_epoch=0.0138]
Validating: 0it [00:00, ?it/s]
Validating:  58%|█████▊    | 22/38 [00:04<00:02,  5.71it/s]


Epoch 24:  80%|███████▉  | 149/187 [01:00<00:15,  2.47it/s, loss=0.014, val_loss_step=0.0176, train_loss_step=0.0136, val_loss_epoch=0.0168, train_loss_epoch=0.0138]
Validating: 0it [00:00, ?it/s]
Epoch 24: 100%|██████████| 187/187 [01:07<00:00,  2.75it/s, loss=0.014, val_loss_step=0.0172, train_loss_step=0.0136, val_loss_epoch=0.0172, train_loss_epoch=0.0138]
Test iterations: 69
Testing:  99%|█████████▊| 68/69 [00:12<00:00,  5.46it/s]Logits: tensor([[ -4.3711,  -5.8125,  -8.4453,  ...,  -8.4688,  -8.5078,  -6.4492],
        [ -5.9961,  -5.0234,  -9.3672,  ...,  -9.7500, -10.7344,  -7.6406],
        [-12.4062, -12.9062,  -9.2500,  ..., -11.6641, -12.6953, -11.7109],
        ...,
        [ -5.1094,  -6.1406,  -8.0391,  ...,  -8.0859,  -7.9961,  -6.4688],
        [ -5.3867,  -5.7227,  -8.0156,  ...,  -7.8906,  -8.6797,  -6.1289],
        [ -6.0820,  -6.9961,  -7.1680,  ...,  -7.5625,  -7.1562,  -6.4375]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[1.248e-02 2.981e-03 2.

# Predict

In [37]:
def run_predict(confFitting, param, test, target, fold, seed):
    
    seed_everything(seed)
    
    x_test = test[confFitting["feature_cols"]]
    
    #データセットをイメージ化するトランスフォーマー。
    #ここでLogScaleも実施。
    all_scaler, all_it, test = PreprocessingLoadTransform(param, x_test, fold, seed)
    x_test = x_test.values
    
    #model class 定義
    model = MoAEfficientNet.load_from_checkpoint(
        checkpoint_path=f"{SAVEMODEL}/model{model_type}_SEED{seed}_FOLD{fold}.ckpt",
        training_set=(None, None),  # tuple
        valid_set=(None, None),  # tuple
        test_set=x_test, #予測用のデータセット
        transformer=all_it,
        drop_rate=drop_rate,
        drop_connect_rate=drop_connect_rate,
        fc_size=fc_size,
        weight_init='goog')
    
    model.freeze()
    model.eval()
    
    trainer = Trainer(
        logger=False,
        gpus=gpus,
        distributed_backend="dp",  # multiple-gpus, 1 machine
        precision=16,
        benchmark=False,
        deterministic=True)
    
    output = trainer.test(model, verbose=False)[0]
    predictions = output["pred_probs"]
    
    return predictions


In [38]:
def run_k_fold_predict(confFitting, test, target, param, Tester, NFOLDS, seed):
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    for fold in range(NFOLDS):
        if Tester:
            print('=' * 20, 'Fold', fold, '=' * 20)
        pred_ = run_predict(confFitting, param, test, target, fold, seed)
        
        predictions += pred_ / NFOLDS
        
    return predictions

In [39]:
def SubmitPredict(confFitting, predictions, test, prefix):
    test[confFitting["target_cols"]] = predictions
    sub = sample_submission.drop(columns=confFitting["target_cols"]).merge(test[['sig_id']+confFitting["target_cols"]], on='sig_id', how='left').fillna(0)
    sub.to_csv(f'{SUBMIT}{prefix}submission.csv', index=False)

    print("sub.shape" + str(sub.shape))
    
    return

In [40]:
def Predict(param):
    #Tester(True/False)
    Tester = False
    
    #Preprocessing Data
    train, test = preprocessing(param, trainFeature, testFeature, trainTargetScored)
    
    #CV folds
    folds = CV_folds_drug_id(train, trainTargetScored)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, test, trainTargetScored, folds)
    
    # Averaging on multiple SEEDS
    SEED = SEED = [42]
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        predictions_ = run_k_fold_predict(confFitting, test, trainTargetScored, param, Tester, NFOLDS, seed)
        predictions += predictions_ / len(SEED)
    
    # 課題提出
    prefix = "Pytorch"
    SubmitPredict(confFitting, predictions, test, prefix)
    
    return

In [41]:
%%time
Predict(param_space)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Test iterations: 63
Testing: 100%|██████████| 63/63 [00:10<00:00,  6.53it/s]Logits: tensor([[-15.7344, -14.8359, -17.3281,  ..., -17.9219, -16.1562, -17.4219],
        [-13.6797, -14.5938, -14.9922,  ..., -16.1250, -14.4922, -15.0312],
        [-19.3125, -18.2188, -20.1562,  ..., -20.3750, -19.9531, -20.2188],
        ...,
        [-10.2500, -10.4062, -11.2578,  ..., -10.8906, -11.4766, -10.8203],
        [-18.6250, -18.5312, -18.2500,  ..., -21.0781, -16.8750, -19.2812],
        [-16.1562, -15.8750, -17.6250,  ..., -19.2969, -16.0000, -18.0781]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[1.192e-07 3.576e-07 5.960e-08 ... 0.000e+00 1.192e-07 0.000e+00]
 [1.132e-06 4.768e-07 2.980e-07 ... 1.192e-07 5.364e-07 2.980e-07]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [3.535e-05 3.022e-05 1.293e-05 ... 1.866e-05 1.037e-05 1.997e-05]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 5.960e-08 0.000e+00]
 [1.192e-07 1.192e-07 0.000e+00 ... 0.000e+00 1

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Test iterations: 63
Testing: 100%|██████████| 63/63 [00:10<00:00,  6.53it/s]Logits: tensor([[-15.0547, -15.7578, -12.0859,  ..., -12.2188, -12.5859, -13.0078],
        [-16.0938, -16.1562, -13.0469,  ..., -13.8203, -15.1719, -14.5391],
        [-14.6484, -15.2109, -11.4062,  ..., -10.8281, -12.4844, -11.7656],
        ...,
        [-13.9922, -14.5781, -11.5078,  ..., -11.1797, -12.4609, -12.1250],
        [-13.8984, -13.3203, -13.1875,  ..., -12.8906, -15.9922, -13.2656],
        [-15.4766, -15.9297, -12.3594,  ..., -12.0078, -14.1016, -13.1016]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[2.980e-07 1.192e-07 5.662e-06 ... 4.947e-06 3.397e-06 2.265e-06]
 [1.192e-07 1.192e-07 2.146e-06 ... 1.013e-06 2.384e-07 4.768e-07]
 [4.172e-07 2.384e-07 1.115e-05 ... 1.985e-05 3.815e-06 7.749e-06]
 ...
 [8.345e-07 4.768e-07 1.007e-05 ... 1.395e-05 3.874e-06 5.424e-06]
 [8.941e-07 1.669e-06 1.848e-06 ... 2.503e-06 1.192e-07 1.729e-06]
 [1.788e-07 1.192e-07 4.292e-06 ... 6.080e-06 7

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Test iterations: 63
Testing: 100%|██████████| 63/63 [00:10<00:00,  6.54it/s]Logits: tensor([[-20.0781, -16.0938, -16.1875,  ..., -18.3906, -16.3906, -18.0938],
        [-15.5547, -12.6484, -11.4375,  ..., -13.7656, -10.3750, -13.2656],
        [-16.9531, -14.4688, -13.7422,  ..., -14.7734, -12.9453, -14.3359],
        ...,
        [-14.9297, -11.9688, -11.4219,  ..., -13.3125, -10.9688, -13.0234],
        [-11.8516,  -9.5391,  -9.6797,  ..., -10.5391,  -9.6250, -10.4219],
        [-19.9844, -15.2500, -17.4062,  ..., -18.9375, -19.5469, -19.1250]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[0.000e+00 1.192e-07 1.192e-07 ... 0.000e+00 5.960e-08 0.000e+00]
 [1.788e-07 3.219e-06 1.079e-05 ... 1.073e-06 3.117e-05 1.729e-06]
 [5.960e-08 5.364e-07 1.073e-06 ... 3.576e-07 2.384e-06 5.960e-07]
 ...
 [3.576e-07 6.318e-06 1.097e-05 ... 1.669e-06 1.723e-05 2.205e-06]
 [7.153e-06 7.200e-05 6.253e-05 ... 2.646e-05 6.604e-05 2.980e-05]
 [0.000e+00 2.384e-07 0.000e+00 ... 0.000e+00 0

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Test iterations: 63
Testing: 100%|██████████| 63/63 [00:10<00:00,  6.52it/s]Logits: tensor([[ -9.9531, -12.0391, -13.5547,  ..., -11.9766, -11.8594, -13.6875],
        [-10.7891, -13.7422, -13.2031,  ..., -10.6484, -12.3984, -11.2891],
        [-10.8203, -12.4297, -13.8281,  ..., -13.0156, -12.8672, -14.2734],
        ...,
        [ -9.9297, -12.1641, -12.8594,  ..., -11.1875, -11.3906, -12.6250],
        [-10.4141, -12.7031, -15.2344,  ..., -13.5391, -13.1641, -15.8125],
        [ -8.3359, -10.4219, -12.0859,  ..., -10.4141, -10.7656, -11.8125]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[4.756e-05 5.901e-06 1.311e-06 ... 6.318e-06 7.093e-06 1.132e-06]
 [2.062e-05 1.073e-06 1.848e-06 ... 2.372e-05 4.113e-06 1.252e-05]
 [1.997e-05 3.994e-06 1.013e-06 ... 2.205e-06 2.563e-06 6.557e-07]
 ...
 [4.870e-05 5.186e-06 2.623e-06 ... 1.383e-05 1.132e-05 3.278e-06]
 [2.998e-05 3.040e-06 2.384e-07 ... 1.311e-06 1.907e-06 1.192e-07]
 [2.397e-04 2.980e-05 5.662e-06 ... 2.998e-05 2

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Test iterations: 63
Testing: 100%|██████████| 63/63 [00:10<00:00,  6.51it/s]Logits: tensor([[-17.2969, -17.7812, -20.1719,  ..., -21.8438, -20.0469, -20.7969],
        [ -9.7500, -10.5938, -12.7891,  ..., -13.1094, -12.6484, -12.6484],
        [-14.9062, -15.6172, -18.0156,  ..., -19.3594, -17.8906, -18.5156],
        ...,
        [-13.8359, -14.7422, -17.0469,  ..., -17.4062, -17.0781, -16.6562],
        [-10.4531, -11.5859, -13.3828,  ..., -13.5703, -13.5781, -13.4922],
        [ -9.9766, -11.4062, -13.8906,  ..., -13.4531, -13.4219, -12.5000]],
       device='cuda:0', dtype=torch.float16)
Predictions:  [[5.960e-08 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [5.829e-05 2.509e-05 2.801e-06 ... 2.027e-06 3.219e-06 3.219e-06]
 [3.576e-07 1.788e-07 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [9.537e-07 4.172e-07 5.960e-08 ... 0.000e+00 5.960e-08 5.960e-08]
 [2.885e-05 9.298e-06 1.550e-06 ... 1.252e-06 1.252e-06 1.371e-06]
 [4.649e-05 1.115e-05 9.537e-07 ... 1.431e-06 1

# Hyperparameter Tuning

In [29]:
#hyperopt
from hyperopt import fmin, tpe, hp, rand, Trials

In [30]:
def HOptExec(param):
    #Tester(True/False)
    Tester = False
    
    #Preprocessing Data
    train, test, target = preprocessing(param, trainFeature, testFeature, trainTargetScored)
    
    #CV folds
    folds = CV_folds(train, target)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, test, target, folds)
    
    # Averaging on multiple SEEDS
    SEED = [0, 1, 2, 3 ,4, 5]
    oof = np.zeros((len(train), confFitting["num_targets"]))
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        oof_, predictions_ = run_k_fold(Tester, NFOLDS, seed, param,
                                       folds, train, test, target, confFitting)
        oof += oof_ / len(SEED)
        predictions += predictions_ / len(SEED)
    
    #CV 評価
    score = CV_Evaluation(confFitting, oof, train, target)
    
    # 課題提出
    #Submit(confFitting, predictions, test)
    
    return score

In [ ]:
%%time

param_space = {'hidden_size1': 512, 
               'hidden_size2': 512, 
               'dropOutRate1': 0.20393004966355735, 
               'dropOutRate2': 0.39170486751620137,
               'rankGauss_n_quantiles': 488.0393350201078,
               'leakyReluSlope': hp.uniform('leakyReluSlope', 1e-3, 1e-1),
              }

trials = Trials()

hopt = fmin(fn = HOptExec, 
            space = param_space, 
            algo = tpe.suggest, 
            max_evals = 15, 
            #timeout = 8.9 * 60 * 60, 
            trials = trials, 
           )

print(hopt)

CV log_loss:                                          
0.014981391207012364                                  
CV log_loss:                                                                         
0.01504250432043703                                                                  
CV log_loss:                                                                         
0.015004835293169368                                                                 
CV log_loss:                                                                         
0.015002514832957038                                                                 
CV log_loss:                                                                         
0.015008986227264749                                                                 
CV log_loss:                                                                         
0.014993115273980633                                                                   
CV log_loss:                